In [4]:
import json
import csv

input_file = "queries.jsonl"   # your JSONL file
output_file = "queries.csv"   # desired CSV file

with open(input_file, "r", encoding="utf-8") as f_in, \
     open(output_file, "w", newline="", encoding="utf-8") as f_out:
    
    writer = csv.writer(f_out)
    writer.writerow(["id", "text"])   # CSV header

    for line in f_in:
        if not line.strip():
            continue
        obj = json.loads(line)
        
        _id = obj.get("_id", "")
        text = obj.get("text", "").replace("\n", " ")  # flatten newlines
        
        writer.writerow([_id, text])

print("CSV saved to:", output_file)


CSV saved to: queries.csv


In [ ]:
import pandas as pd
import openai
import time

# 🔑 Set your OpenAI API key
xy = "sk-proj-"  # <- Replace this with your actual key, or load from env securely


client = openai.OpenAI(api_key=xy)  # Or replace with your key directly


In [11]:

# ✍️ Analysis function
def analyze_video(row):
    note = row["text"] 
    
    prompt = f"""
Convert the following clinical vignette into a JSON onject. Example format:
{{
  "age": 65,
  "gender": "Female",
  
  "conditions": [
    "Non-small cell lung cancer",
    "Breast Cancer",
    "Heart Failure",
    "Chronic Kidney Disease",
    "Respiratory Failure",
    "Liver Failure",
    "Leukemia",
    "Prostate Cancer",
    "Skin Cancer",
    "Cervical Cancer",
    "Bone Cancer"
  ],
  
  "ecog": 1,
  
  "biomarkers": [
    "EGFR",
    "HER2",
    "ALK",
    "KRAS",
    "BRAF",
    "BCR-ABL",
    "FLT3",
    "CD19",
    "ER",
    "PR"
  ],
  
  "history": [
    "History_MI",
    "History_Stroke",
    "Autoimmune_Disease",
    "Prior_Malignancy",
    "HIV",
    "Hepatitis",
    "Pregnancy"
  ],
  
  "labs": {{
    "Creatinine": 1.2,
    "GFR": 55.0,
    "Bilirubin": 0.8,
    "AST": 35.0,
    "ALT": 40.0,
    "INR": 1.1,
    "PSA": 4.5,
    "Testosterone": 300.0,
    "BNP": 150.0,
    "LVEF": 50.0,
    "Platelet_Count": 150,
    "Hemoglobin": 11.5,
    "ANC": 2500
  }},
  
  "prior_lines": 2,
  "days_since_last_treatment": 45
}}Only mention conditions, biomarkers, and lab items that are explicitly mentioned in the vignette. Else it should be null
Only fill in the fields that are explicitly mentioned in the vignette. If a field is not mentioned, do not include it in the JSON.

Now convert the following clinical vignette into JSON:
{note}

Ouptut only json.
"""
    try:
        resp = client.responses.create(
                model="gpt-5",
                input=prompt,
                reasoning={"effort": "minimal"},
                # max_completion_tokens=50,
                text={"verbosity": "low"},
                #  max_output_tokens=70
            )
        # print(prompt)
        # print(resp.output_text)
        return resp.output_text
        # return resp.output_text

    except Exception as e:
        print(f"❌ Error processing row: {e}")
        return "Error"



df=pd.read_csv("queries.csv")

print(analyze_video(df.iloc[34]))

df["json_using_openai"] = df.apply(analyze_video, axis=1)


# # # 💾 Save the combined results to one CSV
df.to_csv("converted_queries_using_openai2.csv", index=False)

print("Combined analysis saved as 'converted_queries_using_openai2.csv'")

{
  "age": 15,
  "gender": "Female",
  "conditions": [
    "Migraine"
  ],
  "labs": {
    "BMI": 21,
    "BP_Systolic": 100,
    "BP_Diastolic": 60
  }
}
Combined analysis saved as 'converted_queries_using_openai2.csv'


In [12]:
import csv
import json

path = "F:/Clinical-Trial-SearchEngine/backend/evaluation/converted_queries_using_openai2.csv"

with open(path, "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for i, row in enumerate(reader, start=2):
        try:
            json.loads(row["json_using_openai"])
        except Exception as e:
            print(f"❌ Error on CSV line {i}: {e}")
            print("Offending JSON:")
            print(row["json_using_openai"])
            break
